In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from typing import *
import copy
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-09-03 16:36:28.797 DEBU

In [2]:
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import *

In [3]:
dataset_orig = read_data()
ne_type = 'PERSON'
dataset = filter_dataset_by_ne_types(dataset_orig, ne_type)

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [4]:
tf.reset_default_graph()
sess = tf.Session()

In [5]:
class ElmoEmbedder():
    def __init__(self, custom_weights=False, weights: list = [], trainable_cells=False, restore_path:str=None):
        self.custom_weights = custom_weights
        self.weights = weights
        module_path = "https://tfhub.dev/google/elmo/{}".format(1 if trainable_cells else 2)
        self.elmo = hub.Module(module_path, trainable=True)
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        self.sess = sess
        elmo_vars = [v.name for v in tf.trainable_variables()]
        elmo_vars_dict = {v.name: v for v in tf.trainable_variables()}
        if restore_path:
            print('Restoring finetuned ELMo params from {}'.format(restore_path))
            saver = tf.train.Saver(elmo_vars_dict)
            saver.restore(sess, restore_path)
        self.elmo_vars = elmo_vars

    def get_tokens_embeddings(self, tokens_input: list, tokens_length:list=None, res_as_dict=False):
        if not tokens_length:
            if isinstance(tokens_input[0], list):
                tokens_length = [len(seq) for seq in tokens_input]
            else:
                tokens_length = len(tokens_input)
        elmo_res = self.elmo(
                        inputs={
                            "tokens": tokens_input,
                            "sequence_len": tokens_length
                        },
                        signature="tokens",
                        as_dict=True)
        embeddings_sum = elmo_res["elmo"]
        word_emb = elmo_res["word_emb"]
        embeddings_layer1 = elmo_res["lstm_outputs1"]
        embeddings_layer2 = elmo_res["lstm_outputs2"]
        embeddings_sum, word_emb, embeddings_layer1, embeddings_layer2 = self.sess.run([embeddings_sum, word_emb, embeddings_layer1, embeddings_layer2])
        word_emb = np.concatenate((word_emb, word_emb), axis=-1)
        if res_as_dict:
            return {'word': word_emb, 'layer1': embeddings_layer1, 'layer2': embeddings_layer2}
        # print(embeddings_sum.shape)
        # print(embeddings_layer1.shape)
        # print(embeddings_layer2.shape)
        embeddings = embeddings_sum
        if self.custom_weights:
            if len(self.weights) == 2:
                embeddings = embeddings_layer1*self.weights[0] + embeddings_layer2*self.weights[1]
            elif len(self.weights) == 3:
                embeddings = word_emb*self.weights[0] + embeddings_layer1*self.weights[1] + embeddings_layer2*self.weights[2]
        return embeddings

In [6]:
class CompositeEmbedder():
    def __init__(self, use_elmo=True, elmo_scale=1., cap_scale=1., use_cap_feat=False, use_glove=False, elmo_params={}):
        self.use_elmo = use_elmo
        self.elmo_scale = elmo_scale
        self.cap_scale = cap_scale
        self.use_cap_feat = use_cap_feat
        self.use_glove = use_glove
        if self.use_elmo:
            self.elmo = ElmoEmbedder(**elmo_params)
        if self.use_cap_feat:
            self.cap_prep = CapitalizationPreprocessor()
        if self.use_glove:
            self.glove = GloVeEmbedder('embeddings/glove.6B/glove.6B.100d.txt', pad_zero=True)
        self.embed_size = self.embed(['hehe']).shape[-1]

    def embed(self, tokens: list, res_as_dict=False):
        if isinstance(tokens[0], str):
            tokens = [tokens]
        # Get ELMo embeddings
        if self.use_elmo:
            tokens_input = add_padding(tokens)
            tokens_length = get_tokens_len(tokens)
            elmo_embed = self.elmo.get_tokens_embeddings(tokens_input, tokens_length)
            embeddings = elmo_embed
            embeddings *= self.elmo_scale
            embed_size = embeddings.shape[-1]
#             print(embeddings.shape)
#             print(embed_size)

        # Use capitalization features
        if self.use_cap_feat:
#             print('Use capitalization features')
            cap_features = self.cap_prep(tokens)*self.cap_scale
    #         print(cap_features)
#             print(cap_features.shape)
            embeddings = np.concatenate((embeddings, cap_features), axis=2)
            embed_size = embeddings.shape[-1]
#             print(embeddings.shape)

        # Use GloVe embeddings
        if self.use_glove:
#             print('Use GloVe')

            glove_embed = self.glove(to_lower_case(tokens))
            glove_embed = np.array(glove_embed)
            if not self.use_elmo:
                embeddings = glove_embed
            else:
                embeddings = np.concatenate((embeddings, glove_embed), axis=2)
            embed_size = embeddings.shape[-1]
#             print(embeddings.shape)

        if res_as_dict:
            return {'elmo': elmo_embed, 'cap': cap_features, 'glove': glove_embed}
        return embeddings

In [7]:
# elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)

In [8]:
# elmo_vars = [v.name for v in tf.trainable_variables()]
# print(elmo_vars)
# elmo_vars_dict = {v.name: v for v in tf.trainable_variables()}
# saver = tf.train.Saver(elmo_vars_dict)
# saver.restore(sess, "./tmp/elmo.ckpt")

In [9]:
embedder2 = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=False, use_glove=False, elmo_params={'trainable_cells': True, 'restore_path': "./model_params/elmo.ckpt"})

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-09-03 16:36:43.839 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/1'.


2018-09-03 16:36:43.842 INFO in 'tensorflow'['tf_logging'] at line 116: Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/1'.


INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/1'.


...

INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/char_embed


2018-09-03 16:37:07.508 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/char_embed


Restoring finetuned ELMo params from ./model_params/elmo.ckpt
INFO:tensorflow:Restoring parameters from ./model_params/elmo.ckpt


2018-09-03 16:37:07.895 INFO in 'tensorflow'['tf_logging'] at line 116: Restoring parameters from ./model_params/elmo.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ./model_params/elmo.ckpt
	 [[Node: save_2/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_2/Const_0_0, save_2/RestoreV2/tensor_names, save_2/RestoreV2/shape_and_slices)]]

Caused by op 'save_2/RestoreV2', defined at:
  File "/home/kostrovskiy/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kostrovskiy/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/traitlets/config/application.

In [ ]:
embedder1 = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=False, use_glove=False, elmo_params={'trainable_cells': True})

In [ ]:
N_EXAMPLES = 10
N_TEST_SENTENCES = 100
MODELS = ['ne_centroid', 'weighted_kNN', 'centroid_kNN', 'svm']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}
USE_CV = False

In [ ]:
np.random.seed(11)
tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
np.random.seed(12)
tokens_test,tags_test = get_data_sample(dataset['test'], min(N_TEST_SENTENCES, len(dataset['test'])))
y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
f1_scores = {}
sims = {}
embedder_names = ['Initial', 'Fine_tuned']
for i, embedder in enumerate([embedder1, embedder2]):
    print('Embedder: {}'.format(embedder_names[i]))
    ner_bin_clf = FewshotNerBinaryClassifier(embedder)
    ner_bin_clf.train_on_batch(tokens_train, tags_train)
    results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
    f1_scores[embedder_names[i]] = {}
    for model in MODELS:
        if results[model].get('sim'):
            sims[embedder_names[i]] = flatten_sim(results[model]['sim'])['cosine']
        pred = results[model]['pred']
        f1 = f1_score(y_test, pred)
        f1_scores[embedder_names[i]][model] = f1

In [ ]:
print(f1_scores)

In [ ]:
plotPDE(sims[embedder_names[0]], y_test, 'Initial ELMo')

In [ ]:
plotPDE(sims[embedder_names[1]], y_test, 'Finetuned ELMo')